# Natal Open Data

In this project we were assigned to use the open data from Natal city available on [http://ckan.imd.ufrn.br/](http://ckan.imd.ufrn.br/) to achieve the following tasks:

- Histogram/bar figure reporting the number of "municipal schools" and "health units".
- Geocoder figure about those units.
- A heatmap figure considering the number of employees in each unit.

## Libraries

Libraries necessary to run this IPython Notebook

In [50]:
!pip install folium
!pip install geocoder
!pip install tqdm

In [51]:
from pathlib import Path
from tqdm import tqdm

import numpy as np
import pandas as pd

import geocoder
import folium
from folium.plugins import HeatMap

from bokeh.charts import Bar, output_notebook, show
from bokeh.models import HoverTool

## Number of municipal schools and health units

Here we used the dataset of public elementary schools and kindergartens* of Natal city in 2014. And the data for health units that has no year specified, but with the info of when it was created we can expect that it's from the year 2016.

[http://ckan.imd.ufrn.br/dataset/quadro-das-escolas-e-cmeis-do-municipio](http://ckan.imd.ufrn.br/dataset/quadro-das-escolas-e-cmeis-do-municipio)
[http://ckan.imd.ufrn.br/dataset/unidades-municipais-de-saude](http://ckan.imd.ufrn.br/dataset/unidades-municipais-de-saude)

`*` described in the code as 'CMEI' - Centro Municipal de Educação Infantil (*Municipal Center for Early Childhood Education*)

### Education data

In [52]:
# Read the education datasets

schools_data = pd.read_csv('escolas-por-regioes-administrativas.csv', encoding = 'utf-8', sep = ';')
cmeis_data = pd.read_csv('cmeis-por-regioes-administrativas.csv', encoding = 'utf-8', sep = ';')

# Normalize NaN values on 'Nº' column
schools_data.loc[:, 'Nº'].fillna('', inplace=True)

# Normalize NaN values on 'ENDEREÇO' column
cmeis_data.loc[:, 'ENDEREÇO'].fillna('', inplace=True)

In [53]:
# Group the data by region

schools_by_region = pd.DataFrame(schools_data.groupby(['Região Administrativa'])['CÓDIGO'].count()).reset_index()
schools_by_region.rename(columns = {
    'Região Administrativa': 'region',
    'CÓDIGO': 'amount'
}, inplace = True)

cmeis_by_region = pd.DataFrame(cmeis_data.groupby(['Região Administrativa'])['CÓDIGO'].count()).reset_index()
cmeis_by_region.rename(columns = {
    'Região Administrativa': 'region',
    'CÓDIGO': 'amount'
}, inplace = True)

In [54]:
# Add collumn to represent the type of the dataset

unit_type = pd.Series('Escola Municipal', index = schools_by_region.index, name = 'type')
schools_by_region = schools_by_region.join(unit_type)

unit_type = pd.Series('CMEI', index = cmeis_by_region.index, name = 'type')
cmeis_by_region = cmeis_by_region.join(unit_type)

### Health units data

Based on the info about the [district and regions of Natal][1] we noted that some district names needed to be normalized or fixed. So, to do that we create a CSV file with the data available on the website cited and used it to update the values on the dataset. Beside that, we find some mistakes on district values, when a value was not a district, but a set of a district, for example. At the end, with the data fixed we used the same CSV to obtain the region of each entry based on district name and created a new dataframe in the format of education's datasets.

[1]: https://pt.wikipedia.org/wiki/Lista_de_bairros_de_Natal_(Rio_Grande_do_Norte)

In [55]:
# Read the health units dataset
health_units_data = pd.read_csv('unidades-de-saude.csv', encoding = 'utf-8', sep = ';')

# Normalize NaN values on 'ENDEREÇO' column
health_units_data.loc[:, 'ENDEREÇO'].fillna('', inplace=True)

# Function to normalize district name
def normalize_district(district):
    district = district.strip().lower()
    words = district.split(' ')
    connectors = ''.join(['da', 'das', 'de', 'do', 'dos'])
    words = list(map(lambda w: w.title() if w not in connectors else w, words))
    return ' '.join(words)

# Normalize text values in column 'BAIRRO'
health_units_data.loc[:, 'BAIRRO'] = health_units_data.loc[:, 'BAIRRO'].apply(normalize_district)

# 'Pq dos Coqueiros' is not a district, it belongs to district 'Nossa Senhora da Apresentação'
# https://pt.wikipedia.org/wiki/Nossa_Senhora_da_Apresenta%C3%A7%C3%A3o_(bairro_de_Natal)
health_units_data.loc[health_units_data['BAIRRO'] == 'Pq. dos Coqueiros', 'BAIRRO'] = 'Nossa Senhora da Apresentação'

# 'Brasília Teimosa' is not a district, it belongs to district 'Santos Reis'
# https://pt.wikipedia.org/wiki/Santos_Reis_(Natal)
health_units_data.loc[health_units_data['BAIRRO'] == 'Brasília Teimosa', 'BAIRRO'] = 'Santos Reis'

# 'Pirangi' is not a district, it belongs to district 'Neópolis'
# https://pt.wikipedia.org/wiki/Ne%C3%B3polis_(Natal)
health_units_data.loc[health_units_data['BAIRRO'] == 'Pirangi', 'BAIRRO'] = 'Neópolis'

# Fix district typos
health_units_data.loc[health_units_data['BAIRRO'].str.contains('presentação'), 'BAIRRO'] = 'Nossa Senhora da Apresentação'
health_units_data.loc[health_units_data['BAIRRO'] == 'Cid. da Esperança', 'BAIRRO'] = 'Cidade da Esperança'
health_units_data.loc[health_units_data['BAIRRO'] == 'Mãe Luiza', 'BAIRRO'] = 'Mãe Luíza'
health_units_data.loc[health_units_data['BAIRRO'] == 'Bairro Nordeste', 'BAIRRO'] = 'Nordeste'
health_units_data.loc[health_units_data['BAIRRO'] == 'Nazaré', 'BAIRRO'] = 'Nossa Senhora de Nazaré'
health_units_data.loc[health_units_data['BAIRRO'] == 'Nova Cidade', 'BAIRRO'] = 'Cidade Nova'

In [56]:
# Add 'Região Administrativa' column to health_units_data based on district_by_regions

district_by_regions = pd.read_csv('bairros-por-regioes-administrativas.csv')

region = pd.Series(None, index = health_units_data.index, name = 'Região Administrativa')
health_units_data = health_units_data.join(region)

for i in health_units_data.index:
    district = health_units_data.loc[i, 'BAIRRO']
    dr_filter = district_by_regions[district_by_regions['district'] == district]
    region = dr_filter.get_value(dr_filter.index[0], 'region')
    health_units_data.loc[i, 'Região Administrativa'] = region

In [57]:
# Group the data by region

health_units_by_region = pd.DataFrame(health_units_data.groupby(['Região Administrativa'])['ORDEM'].count()).reset_index()
health_units_by_region.rename(columns = {
    'Região Administrativa': 'region',
    'ORDEM': 'amount'
}, inplace = True)

In [58]:
# Add collumn to represent the type of the dataset

unit_type = pd.Series('Unidade de Saúde', index = health_units_by_region.index, name = 'type')
health_units_by_region = health_units_by_region.join(unit_type)

In [59]:
# Create a new dataframe using the extracted data

frames = [schools_by_region, cmeis_by_region, health_units_by_region]
units_by_region = pd.concat(frames, ignore_index = True)

### Results

And now we have a figure reporting the total number of elementary schools (*Escolas Municipais*), kindergartens (*CMEIs*) and health units (*Unidades de Saúde*) gruped by regions (*Região Administrativa*) of the city.

In [60]:
p = Bar(units_by_region,
        label='region',
        values='amount',
        group='type',
        title='Total de Escolas Municipais, CMEIs e Unidades de Saúde de Natal agrupados por região',
        legend='top_right',
        xlabel='Região Administrativa',
        ylabel='Total de Unidades',
        plot_width=800, plot_height=400)

tooltip = HoverTool(
            tooltips=[
                ('Estabelecimento', '@type'),
                ('Total de unidades', "@height"),
                ('Zona', '@region')
            ]
        )

p.add_tools(tooltip)

output_notebook()

show(p)

Loading BokehJS ...

## Geolocation of units

Using the three dataframes obtained from the open data of the city, we now add new columns referencing the geolocation data of each entry and calcute its coordinates using address information available. With this new data we plot markers on a map representing the location of each unit.

In [61]:
# Prepare elementary schools geolocation data

schools_data['GEOCODE_INPUT'] = schools_data['ESTABELECIMENTO'] + ', ' + schools_data['BAIRRO'] + ', ' + schools_data['ENDEREÇO'] + ', ' + schools_data['Nº']
schools_data['LAT'], schools_data['LNG'] = [0, 0]
schools_geolocation = schools_data[['ESTABELECIMENTO', 'GEOCODE_INPUT', 'LAT', 'LNG']]

unit_type = pd.Series('Escola Municipal', index = schools_geolocation.index, name = 'TYPE')
schools_geolocation = schools_geolocation.join(unit_type)

schools_geolocation.rename(columns = {
    'ESTABELECIMENTO': 'NAME'
}, inplace = True)

In [62]:
# Prepare kindergartens geolocation data

cmeis_data['GEOCODE_INPUT'] = cmeis_data['ESTABELECIMENTO'] + ', ' + cmeis_data['BAIRRO'] + ', ' + cmeis_data['ENDEREÇO'] + ', ' + cmeis_data['Nº']
cmeis_data['LAT'], cmeis_data['LNG'] = [0, 0]
cmeis_geolocation = cmeis_data[['ESTABELECIMENTO', 'GEOCODE_INPUT', 'LAT', 'LNG']]

unit_type = pd.Series('CMEI', index = cmeis_geolocation.index, name = 'TYPE')
cmeis_geolocation = cmeis_geolocation.join(unit_type)

cmeis_geolocation.rename(columns = {
    'ESTABELECIMENTO': 'NAME'
}, inplace = True)

In [63]:
# Prepare health units geolocation data

health_units_data['GEOCODE_INPUT'] = health_units_data['NOME DO ESTABELECIMENTO'] + ', ' + health_units_data['BAIRRO'] + ', ' + health_units_data['ENDEREÇO']
health_units_data['LAT'], health_units_data['LNG'] = [0, 0]
health_units_geolocation = health_units_data[['NOME DO ESTABELECIMENTO', 'GEOCODE_INPUT', 'LAT', 'LNG']]

unit_type = pd.Series('Unidade de Saúde', index = health_units_geolocation.index, name = 'TYPE')
health_units_geolocation = health_units_geolocation.join(unit_type)

health_units_geolocation.rename(columns = {
    'NOME DO ESTABELECIMENTO': 'NAME'
}, inplace = True)

In [64]:
# Group the dataframes

frames = [schools_geolocation, cmeis_geolocation, health_units_geolocation]
geolocation_df = pd.concat(frames, ignore_index = True)

In [65]:
# Calculate the geolocation info (if we already have done this, only load the CSV)

if Path('units-geolocation.csv').is_file():
    geolocation_df = pd.read_csv('units-geolocation.csv')
else:
    for i in tqdm(range(len(geolocation_df))):
        g = geocoder.google(geolocation_df.loc[i,'GEOCODE_INPUT'])
        geolocation_df.ix[i,'LAT'] = g.lat
        geolocation_df.ix[i,'LNG'] = g.lng

### Results

Show the units on geolocation map:

- Green: Elementary Schools (*Escolas Municipais*)
- Red: Kindergartens (*CMEIs*)
- Blue: Health Units (*Unidades de Saúde*)

In [66]:
f_map = folium.Map(
    location = [-5.791659, -35.228385],
    zoom_start= 12
)

unit_type_colors = {
    'Escola Municipal': 'green',
    'CMEI': 'red',
    'Unidade de Saúde': 'blue'
}

for i in tqdm(range(len(geolocation_df))):
    if not np.isnan(geolocation_df.ix[i,'LAT']) and not np.isnan(geolocation_df.ix[i,'LNG']):
        folium.Marker(
            [float(geolocation_df.ix[i,'LAT']), float(geolocation_df.ix[i,'LNG'])],
            icon = folium.Icon(
                color = unit_type_colors[geolocation_df.ix[i, 'TYPE']],
                icon = 'info-sign'
            ),
            popup = geolocation_df.ix[i, 'NAME']
        ).add_to(f_map)

100%|██████████| 232/232 [00:05<00:00, 46.27it/s]


In [67]:
f_map

## Heatmap of the number of employees in each unit

In this section we should show a heatmap of the number of employees in each unit, but we had some problems with the datasets of health units, where the [data available][1] don´t have a correspondent unit name with the name of the units that were used to generate the dataframe of the previous section. So, we could achieve only the education part of the task.

However, the [education data available][2] for the unit names matches with the data from the `geolocation_df`. With that in mind, using the new datasets we sum the total of employees in each unit and add this info to the geolocation dataframe.

[1]: http://ckan.imd.ufrn.br/dataset/profissionais-das-unidades-de-saude-por-distrito
[2]: http://ckan.imd.ufrn.br/dataset/docentes-por-estabelecimento-de-ensino

In [68]:
# Read the datasets
kindergarten_data = pd.read_csv('funcao-docente-do-ens.-infantil-por-estabelecimento.csv', encoding = 'utf-8', sep = ';')
elementary_school_data = pd.read_csv('funcao-docente-do-ens.-fundamental-por-estabelecimento.csv', encoding = 'utf-8', sep = ';')

# Group the data
frames = [kindergarten_data, elementary_school_data]
school_employees_data = pd.concat(frames, ignore_index = True)

# Sum columns of the data and add it to a new column
school_employees_data['Total'] = school_employees_data.sum(axis=1)

In [69]:
# Add a new column to geolocation dataframe to add employees amount info

employees_amount = pd.Series(0, index = geolocation_df.index, name = 'EMPLOYEES_AMOUNT')
geolocation_df = geolocation_df.join(employees_amount)

In [70]:
# Verify each entry in `geolocation_df` and add the employees amount info

for i in tqdm(range(len(geolocation_df))):
    if not np.isnan(geolocation_df.ix[i,'LAT']) and not np.isnan(geolocation_df.ix[i,'LNG']):
        unit_name = geolocation_df.ix[i,'NAME']
        unit_employees = school_employees_data[school_employees_data['Estabelecimento'] == unit_name]

        if len(unit_employees.index) > 0:
            amount = unit_employees.ix[unit_employees.index[0], 'Total']
            geolocation_df.ix[i,'EMPLOYEES_AMOUNT'] = amount

100%|██████████| 232/232 [00:00<00:00, 971.33it/s]


### Results

Now we can show a heatmap with the elementary schools and kindergartens based on the total of employees in each unit.

In [71]:
# Create a coordinates list with the filtered data and show the heatmap

coordinates = []

for i in range(len(geolocation_df)):
    if all(~np.isnan([geolocation_df.ix[i,'LAT'], geolocation_df.ix[i,'LNG']])) and geolocation_df.ix[i,'EMPLOYEES_AMOUNT'] != 0:
        coordinates.append([geolocation_df.ix[i,'LAT'], geolocation_df.ix[i,'LNG'], geolocation_df.ix[i,'EMPLOYEES_AMOUNT']])
        
f_map = folium.Map(
    location = [-5.791659, -35.228385],
    zoom_start= 12
)

HeatMap(coordinates).add_to(f_map)

f_map

## References

- [http://pandas.pydata.org/pandas-docs/stable/indexing.html](http://pandas.pydata.org/pandas-docs/stable/indexing.html)
- [http://pandas.pydata.org/pandas-docs/stable/merging.html](http://pandas.pydata.org/pandas-docs/stable/merging.html)
- [http://pandas.pydata.org/pandas-docs/stable/groupby.html](http://pandas.pydata.org/pandas-docs/stable/groupby.html)
- [http://www.shanelynn.ie/select-pandas-dataframe-rows-and-columns-using-iloc-loc-and-ix/](http://www.shanelynn.ie/select-pandas-dataframe-rows-and-columns-using-iloc-loc-and-ix/)
- [https://github.com/python-visualization/folium/blob/master/folium/plugins/heat_map.py](https://github.com/python-visualization/folium/blob/master/folium/plugins/heat_map.py)